The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.


In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

import os
os.environ["OPEN_API_KEY"] = "your key"


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def divide(first_int: int, second_int: int) -> int:
    """Divide first integer by second integer."""
    return first_int / second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int

@tool
def subtract(first_int: int, second_int: int) -> int:
    "sbutract two integers."
    return first_int - second_int


subtract.invoke({"first_int": 4, "second_int": 5})

from operator import itemgetter
from typing import Union

from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

model = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], model_name="gpt-4-0125-preview")

tools = [multiply, divide, add, subtract]



In [50]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
# Get the prompt to use 
# https://github.com/hwchase17/langchain-hub?tab=readme-ov-file
#prompt = load_prompt('lc://prompts/conversation/prompt.json')

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(
            variable_name="chat_history",optional=True
        ),
        ("human", "{input}"),
        MessagesPlaceholder(
            variable_name="agent_scratchpad",
        ),
    ]
)
prompt.messages
#prompt.pretty_print()

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [40]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)



In [47]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "ninety divided by three, multiply by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `divide` with `{'first_int': 90, 'second_int': 3}`


30.0
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 30, 'second_int': 15}`


450
Invoking: `multiply` with `{'first_int': 450, 'second_int': 450}`


202500The result of dividing ninety by three, multiplying by the sum of twelve and three, and then squaring the whole result is 202,500.

> Finished chain.


{'input': 'ninety divided by three, multiply by the sum of twelve and three, then square the whole result',
 'output': 'The result of dividing ninety by three, multiplying by the sum of twelve and three, and then squaring the whole result is 202,500.'}